#Long-Short Equity

In [0]:
#if you are using google drive like me
from google.colab import drive
drive.mount('/content/drive/')

In [0]:
cd "/content/drive/My Drive/go_beyond/project/project_1"

In [0]:
#helper file with functions created in stocks returns analysis
%run helper_returns.ipynb

In [0]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import math

#remove pandas warning 
import pandas.plotting._converter as pandacnv
pandacnv.register()

##Loading Data 

You can see the analysis of this data in stocks returns file. :)

All funcitions have been explained [here](https://https://nbviewer.jupyter.org/github/victormenuzzo/Quantitative-Analysis/blob/master/part1/stocks_returns.ipynb)

In [0]:
df = pd.read_csv('eod-quotemedia.csv', parse_dates=['date'], index_col=False)
close = df.reset_index().pivot(index='date', columns='ticker', values='adj_close')

#resample daily -> monthly
df_monthly = resample_prices(close)

#raws returns
df_rawRet = compute_rawReturn(df_monthly)

#logs returns 
df_logRet = compute_logReturn(df_monthly)

##Trading Signal